In [1]:
import pandas as pd
import numpy as np
from langdetect import detect
import re

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
comments = pd.read_csv('../data/comments_lang.csv')
comments_en = comments[comments.lang == 'en']
vect = CountVectorizer(ngram_range = (1,1), analyzer = 'word',
                       stop_words = 'english',
                       max_features = 10000,
                       min_df = 2, max_df = 0.95).fit(comments_en.Review)

In [3]:
pw = list(vect.vocabulary_.keys())

In [4]:
import enchant
c = enchant.Dict("en_UK")
def check_spelling(text):
    if not c.check(text):
        suggestions = list(set(c.suggest(text)).intersection(set(pw)))
        if len(suggestions)>0:
            res = suggestions[0]
        else:
            res = text
    else:
        res = text
    return res

In [12]:
check_spelling('gud')

'god'

In [6]:
labeled1 = pd.read_excel('manual_labels.xlsx')
labeled2 = pd.read_excel('manual_labels2.xlsx')
labeled3 = pd.read_excel('manual_labels3.xlsx')
labeled4 = pd.read_excel('manual_labels4.xlsx')
labeled5 = pd.read_excel('manual_labels5.xlsx')

In [7]:
labeled5.label.value_counts()

4    621
2    172
5     89
0     82
3     25
1     11
Name: label, dtype: int64

In [8]:
labeled2.shape

(1000, 5)

In [9]:
labeled1.label.value_counts()

4    624
2    203
5     74
0     60
3     31
1      8
Name: label, dtype: int64

In [10]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
def clean_comment(text):
    wnl = WordNetLemmatizer()
    deacc = re.sub(r'\W',' ', text)
    tokens = word_tokenize(deacc)
    res = ''
    for t in tokens:
        res += wnl.lemmatize(t)+' '
    return res
def get_tokens(text):
    wnl = WordNetLemmatizer()
    deacc = re.sub(r'\W',' ', text)
    tokens = word_tokenize(deacc)
    return tokens

In [13]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import nltk
from nltk.corpus import wordnet
from emoji.unicode_codes import UNICODE_EMOJI
import emoji
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

def clean_comment(text):
    wnl = WordNetLemmatizer()
    deacc = re.sub(r'\!',' exclamation_point ', text)
    tokens = word_tokenize(deacc)
    tags = nltk.pos_tag(tokens)
    processed = []
    for (word, tag) in tags:
        wn_tag = get_wordnet_pos(tag)
        if wn_tag!='':
            processed.append(wnl.lemmatize(word,wn_tag))
        else:
            processed.append(wnl.lemmatize(check_spelling(word)))
    res = ' '.join(processed)
    return res

In [14]:
labeled = pd.concat([labeled1, labeled2, labeled3, labeled4, labeled5],axis = 0)

In [15]:
labeled.loc[:,'cleaned'] = labeled.Review.apply(clean_comment)

In [16]:
labeled['tokens'] = labeled.Review.apply(get_tokens)

In [17]:
labeled_long = labeled[labeled.tokens.apply(len)>6]
labeled_neg = labeled[labeled.label!=4]

In [18]:
#comments_en['cleaned'] = comments_en.Review.apply(clean_comment)

In [19]:
#comments_en.to_csv('comments_en_cleaned.csv')

In [20]:
comments_en = pd.read_csv('comments_en_cleaned.csv')

In [21]:
from keras.preprocessing.text import Tokenizer
from keras.layers import TimeDistributed
from keras.models import load_model
import re
import keras.backend as K
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras import regularizers
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Flatten
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from keras.layers.recurrent import LSTM

Using TensorFlow backend.


In [23]:
t = Tokenizer()

#rds = corpus[rd.tokens.apply(len)>5]
t = Tokenizer()
t.fit_on_texts(comments_en.cleaned.tolist())
vocab_size = len(t.word_index) + 1

In [25]:
encoded_docs = t.texts_to_sequences(labeled.cleaned)
max_length = labeled.tokens.apply(len).max()
feats = encoded_docs
labels = to_categorical(labeled.label)
X_train, X_test, y_train, y_test = train_test_split(feats, labels, test_size=0.2)
X_train = sequence.pad_sequences(X_train, maxlen=max_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_length)

In [26]:
model = Sequential()

model.add(Embedding(vocab_size, 150, 
                    input_length=max_length,
                   embeddings_regularizer = regularizers.l2(1e-3)))
model.add(Dropout(0.2))
model.add(Conv1D(filters=100, kernel_size=25, padding='same', activation='sigmoid'))
model.add(Conv1D(filters=25, kernel_size=25, padding='same', activation='sigmoid'))
model.add(MaxPooling1D(pool_size=5))
#model.add(Flatten())
model.add(Dropout(0.2))
model.add(LSTM(10, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [27]:
class_weight = compute_class_weight('balanced'
                                               ,[0,1,2,3,4,5]
                                               ,labeled.label.apply(int).tolist())

In [ ]:
checkpointer = ModelCheckpoint(filepath='weights.hdf5', verbose=1, save_best_only=True)
model.fit(X_train, y_train, epochs=200, batch_size=500,
          validation_data = [X_test,y_test],
          callbacks=[checkpointer],class_weight = class_weight)

Train on 4000 samples, validate on 1000 samples
Epoch 1/200
3500/4000 [=========================>....] - ETA: 8s - loss: 3.0585 - categorical_accuracy: 0.1989 
Epoch 00001: val_loss improved from inf to 2.42423, saving model to weights.hdf5
4000/4000 [==============================] - 71s 18ms/step - loss: 2.9940 - categorical_accuracy: 0.2255 - val_loss: 2.4242 - val_categorical_accuracy: 0.5910
Epoch 2/200
3500/4000 [=========================>....] - ETA: 8s - loss: 2.2414 - categorical_accuracy: 0.4857 
Epoch 00002: val_loss improved from 2.42423 to 1.87397, saving model to weights.hdf5
4000/4000 [==============================] - 71s 18ms/step - loss: 2.2060 - categorical_accuracy: 0.4877 - val_loss: 1.8740 - val_categorical_accuracy: 0.5910
Epoch 3/200
3500/4000 [=========================>....] - ETA: 8s - loss: 1.7983 - categorical_accuracy: 0.5383 
Epoch 00003: val_loss improved from 1.87397 to 1.58254, saving model to weights.hdf5
4000/4000 [==============================] - 70

3500/4000 [=========================>....] - ETA: 7s - loss: 1.2501 - categorical_accuracy: 0.5894 
Epoch 00024: val_loss improved from 1.21925 to 1.21678, saving model to weights.hdf5
4000/4000 [==============================] - 67s 17ms/step - loss: 1.2374 - categorical_accuracy: 0.5972 - val_loss: 1.2168 - val_categorical_accuracy: 0.5910
Epoch 25/200
3500/4000 [=========================>....] - ETA: 7s - loss: 1.2401 - categorical_accuracy: 0.5977 
Epoch 00025: val_loss improved from 1.21678 to 1.21289, saving model to weights.hdf5
4000/4000 [==============================] - 67s 17ms/step - loss: 1.2397 - categorical_accuracy: 0.5983 - val_loss: 1.2129 - val_categorical_accuracy: 0.5910
Epoch 26/200
3500/4000 [=========================>....] - ETA: 7s - loss: 1.2391 - categorical_accuracy: 0.5971 
Epoch 00026: val_loss improved from 1.21289 to 1.20979, saving model to weights.hdf5
4000/4000 [==============================] - 67s 17ms/step - loss: 1.2294 - categorical_accuracy: 0.5

Epoch 48/200
3500/4000 [=========================>....] - ETA: 7s - loss: 0.7720 - categorical_accuracy: 0.7414 
Epoch 00048: val_loss did not improve
4000/4000 [==============================] - 67s 17ms/step - loss: 0.7647 - categorical_accuracy: 0.7440 - val_loss: 0.8617 - val_categorical_accuracy: 0.7260
Epoch 49/200
3500/4000 [=========================>....] - ETA: 7s - loss: 0.7386 - categorical_accuracy: 0.7483 
Epoch 00049: val_loss did not improve
4000/4000 [==============================] - 67s 17ms/step - loss: 0.7417 - categorical_accuracy: 0.7460 - val_loss: 0.8467 - val_categorical_accuracy: 0.7250
Epoch 50/200
3500/4000 [=========================>....] - ETA: 7s - loss: 0.7440 - categorical_accuracy: 0.7457 
Epoch 00050: val_loss did not improve
4000/4000 [==============================] - 67s 17ms/step - loss: 0.7360 - categorical_accuracy: 0.7480 - val_loss: 0.8450 - val_categorical_accuracy: 0.7230
Epoch 51/200
3500/4000 [=========================>....] - ETA: 7s - lo

Epoch 75/200
3500/4000 [=========================>....] - ETA: 7s - loss: 0.6170 - categorical_accuracy: 0.7966 
Epoch 00075: val_loss did not improve
4000/4000 [==============================] - 67s 17ms/step - loss: 0.6111 - categorical_accuracy: 0.7988 - val_loss: 0.9648 - val_categorical_accuracy: 0.7230
Epoch 76/200
3500/4000 [=========================>....] - ETA: 7s - loss: 0.5892 - categorical_accuracy: 0.8069 
Epoch 00076: val_loss did not improve
4000/4000 [==============================] - 66s 16ms/step - loss: 0.5989 - categorical_accuracy: 0.8055 - val_loss: 1.0358 - val_categorical_accuracy: 0.7190
Epoch 77/200
3500/4000 [=========================>....] - ETA: 7s - loss: 0.5949 - categorical_accuracy: 0.8031 
Epoch 00077: val_loss did not improve
4000/4000 [==============================] - 66s 16ms/step - loss: 0.6075 - categorical_accuracy: 0.8020 - val_loss: 1.0000 - val_categorical_accuracy: 0.7200
Epoch 78/200
3500/4000 [=========================>....] - ETA: 7s - lo

Epoch 102/200
3500/4000 [=========================>....] - ETA: 16s - loss: 0.5313 - categorical_accuracy: 0.8394
Epoch 00102: val_loss did not improve
4000/4000 [==============================] - 130s 33ms/step - loss: 0.5343 - categorical_accuracy: 0.8385 - val_loss: 1.2133 - val_categorical_accuracy: 0.7170
Epoch 103/200
3500/4000 [=========================>....] - ETA: 7s - loss: 0.5226 - categorical_accuracy: 0.8383 
Epoch 00103: val_loss did not improve
4000/4000 [==============================] - 69s 17ms/step - loss: 0.5233 - categorical_accuracy: 0.8362 - val_loss: 1.2058 - val_categorical_accuracy: 0.7150
Epoch 104/200
3500/4000 [=========================>....] - ETA: 8s - loss: 0.5170 - categorical_accuracy: 0.8403 
Epoch 00104: val_loss did not improve
4000/4000 [==============================] - 72s 18ms/step - loss: 0.5146 - categorical_accuracy: 0.8390 - val_loss: 1.1975 - val_categorical_accuracy: 0.7240
Epoch 105/200
3500/4000 [=========================>....] - ETA: 10

In [132]:
def eval_model(y_train,y_test,y_train_pred,y_test_pred):
    
    class_names = ['unknown',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    
    class_names_b = ['neg', 'pos']
    print('train scores\n')
    print(classification_report(y_train, y_train_pred, target_names = class_names))
    print('test scores\n')
    print(classification_report(y_test, y_test_pred, target_names = class_names))

In [141]:
from sklearn.metrics import f1_score,roc_auc_score,accuracy_score, classification_report
def eval_network(input_text, model = model):
    cleaned_text = clean_comment(input_text)
    class_names = ['Other',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    seq = t.texts_to_sequences([cleaned_text])
    padded_sequence = sequence.pad_sequences(seq, maxlen=max_length)
    prediction = model.predict(padded_sequence)
    #print(class_names[prediction[0]])
    return np.argmax(class_weight*prediction[0])

def val_score(model):
    class_names = ['Other',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']    
    val_en = pd.read_excel('validation_en.xlsx')
    y_true = []
    y_pred = []
    for i in range(0,6):
        y_true.append([i]*10)
        y_pred.append(val_en.iloc[:,i].apply(eval_network))
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    print(classification_report(y_true, y_pred, target_names = class_names))
    print('model accuracy %1.4f'%(accuracy_score(y_true, y_pred)))
    return y_true,y_pred
y_true,y_pred = val_score(model)

                  precision    recall  f1-score   support

           Other       0.80      0.80      0.80        10
           Crash       0.33      0.20      0.25        10
Balance problems       0.00      0.00      0.00        10
 Synchronization       0.29      0.80      0.42        10
        Positive       0.60      0.60      0.60        10
             Bug       0.00      0.00      0.00        10

     avg / total       0.34      0.40      0.35        60

model accuracy 0.4000


/Users/dns/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [142]:
eval_network('this dum gaem crashes every time i launch it')

5

In [143]:
eval_network('those new weapons are so dam op')

1

In [144]:
eval_network('The game glitched and all of my trophies and guns are now lost')

1

In [145]:
eval_network('Cool!')

4

In [146]:
eval_network('This game is haard to control')

0

In [147]:
eval_network('Mucho gusto')

0